In [ ]:
import json

import chess
from IPython.core.display import HTML
with open('style.html', 'r') as file:
     css = file.read()
HTML(css)

In [ ]:
%run Util/00_imports.ipynb
%run Util/01_functions.ipynb

In [ ]:
# Global variables:

USED_BOARDS = set()
PIECE_LIST = [chess.Piece.from_symbol("K"), chess.Piece.from_symbol("k"), chess.Piece.from_symbol("Q")]
USER_WANTS_PAWN = chess.Piece.from_symbol("P") in PIECE_LIST
FILENAME = "test_18_05"

# Überlegung im Vergleich zu vorherigen Version:
* Muss der Nutzer wirklich einen Namen der Datei angeben? Können wir diesen nicht generisch erstellen?

# Voraussetzungen:
* Nutzer ist immer weiß

# Strukturüberlegung:
* Zentrale used_boards -> Müssen nicht übergeben werden
* Uniques werden gespeichert anstatt kompletter s_n Menge
* Angabe des Nutzers der aktuellen Situation
* Nutzer startet Berechnung:

# Benötigte Funktionen:

* Spiegelungen aus Uniques generieren; gen_mirroring(uniques):
    * Über Uniques iterieren
        * Über Spiegelarten iterieren
            * Spiegelung durchführen (ohne Objekterstellung in Integer-Schreibweise spiegeln)
            * Spiegelungen in Menge zusammenfassen
    * Menge zurückgeben

# Berechnung der Endspieldatenbank

Wie in Notebook ``01_chess_introduction`` bereits erklärt, ist ein Schachspiel gewonnen, wenn die gegnerische Figur mattgesetzt wurde. 

> TODO: So umschreiben, dass P auch direkt als Menge der Figuren angesehen werden kann. ?

Bei einer geringen Anzahl $n$ an Figuren (in der Menge $P$) im Spielzustand lassen sich alle möglichen Positionen berechnen.
Ausgehend von allen Schachmatt-Spielsituationen, können Zugfolgen bestimmt werden, welche zu einer Spielsituation $n$ Halbzüge vom Sieg entfernt führen.

Im weiteren Verlauf werden folgende Definitionen verwendet:
* $\mathtt{board.pieces}$: Liste der Figuren, welche in einem Zustand vorhanden sind.
* $\mathtt{valid\_boards}$: Alle Zustände des Schachspiels, die gegen keine Regeln verstoßen.  
  Relevante Regeln sind:
  * Zwei Könige auf dem Spielfeld
  * Die Könige nicht auf benachbarten Spielfeldern
  * Keine Bauern auf der 1. und 8. Zeile  
* $\mathtt{won\_boards}$: Alle Zustände des Schachspiels, in denen ein Spieler gewonnen hat.
* $\mathtt{previous\_states(b)}$: Alle Zustände, aus denen durch Ausführen eines einzelnen Zuges der Zustand $b$ erreicht werden kann.

Seien alle möglichen (validen) Kombinationen von Positionen der Figuren $P$ die Menge $S$.  
Für $S$ gilt:  
* $board \in S \implies \forall p \in P : p \in \mathtt{board.pieces}$
* $board \in S \implies board \in \mathtt{valid\_boards}$

> TODO: Überlegung, ob wir das wirklich so als "Hinweis" schreiben. Vielleicht doch aktiv als Absatz schreiben

Hinweis: in einem Regulärem-Schachspiel müssen auch Stellungen mit geschlagenen Figuren betrachtet werden.  
Aus der Aufgabenstellung (Aufgeführt in `readme.md`) folgt jedoch, dass in den Spielsituationen keine Figur geschlagen und das Spiel noch gewonnen werden kann.

Aus der Menge $S$ lassen sich Zustände auswählen, welche $n$ Züge vom Sieg entfernt sind. 
Diese Zustände lassen sich in $S_n$ zusammenfassen. Ist das Spiel gewonnen, verbleiben 0 Züge bis zum Sieg.  
Für alle diese Zustände, in denen ein Spieler mattgesetzt ist, gilt:  
  
$$board \in S_0 \implies board \in won\_boards \land board \in S$$  

Aus dieser Definition können induktiv die verbleibenden $S_n$ hergeleitet werden:  
* $board \in S_{n+1} \iff board \in S \land \exists b \in S_n: board \in \mathtt{previous\_states(b)}$
* $board \in S_{2n} \iff board \in S \land \forall b \in \mathtt{next\_states(board)}: b \in S_{m} \land m < 2n$

Für die Berechnungen in diesem Notebook gilt, da für den schwarzen Spieler immer nur der König auf dem Feld steht, weiter Folgendes:
$board.turn$: Der Spieler, welcher am Zug ist.
> Der Spieler, welcher am Zug ist. -> Kein Satz

* $\forall b \in S_{2n+1} : b.turn = schwarz$
* $\forall b \in S_{2n} : b.turn = weiß$

Dieses Notebook wird zur Berechnung der $S_n$ Mengen verwendet. Diese werden benötigt, um letztendlich ein Schach-Endspiel lösen zu können. 

## Ein Hinweis zur effizienten Ergebnisverwaltung
Im Verlauf der Berechnung muss mehrfach überprüft werden, ob eine Stellung bereits bekannt und einem $S_n$ zugeordnet ist.
Da der Abgleich mit einer Liste in Python ineffizient ist, findet dieser Abgleich mit Mengen statt.
Mengen werden in Python als Hash-Tabellen umgesetzt und haben damit eine Zeitkomplexität bei der Überprüfung, ob sie ein bestimmtes Element enthalten von $\mathcal{O}(1)$.
`board` Objekte der `chess` Library sind jedoch nicht "Hashbar". Im Sinne der in dieser Arbeit getätigten Berechnungen reichen die Informationen über die Stellung der Figuren und dem Spieler, welcher am Zug ist, aus. Es wird daher für die Verwendung in Python Mengen mit einer Integer-Repräsentation der Stellungen gearbeitet. 

> TODO: Besser erklären + Überlegung schon vorher in 01_functions erklären, weil da Funktionen zu definiert werden.

Diese Repräsentationen werden bestimmt, indem die Positionen der Figuren, welche in der globalen Variable `PIECE_LIST` aufgeführt werden, binär zusammengefügt werden. Weiter wird ein Bit gesetzt, um darzustellen, welcher Spieler am Zug ist.  

Eine Erklärung der Funktionen `to_integer()` und `to_board()` befinden sich im Notebook `/Util/01_functions.ipynb`.

> TODO: Beispiel einfügen

> TODO: Rückwärts besser erläutern? (Unsicher, da TODO schon davor hier stand)

Um die Effizienz weiter zu steigern, berechnet dieses Notebook nicht alle Stellungen $S$ und entfernt daraus die Stellungen für ein $S_n$ wie in der Aufgabenstellung beschrieben.
Stattdessen werden alle bekannten Stellungen in `USED_BOARDS` gespeichert.
Dopplungen werden also nicht vermieden indem Stellungen aus einer großen Liste entfernt werden, sondern eine Liste der entfernten Stellungen geführt und neue Stellungen mit dieser abgeglichen.

## Ein Hinweis zu Spiegelungen
In diesem Notebook werden Spiegelungen der Stellungen verwendet. Die technische Umsetzung 
dieser Spiegelungen werden im Verlauf des Dokuments erklärt, an dieser Stelle soll lediglich
eine Einführung in die Theorie hinter dem Spiegeln von Stellungen erklärt werden.

Durch die zuvor erklärten Bewegungsmuster der Figuren sind Schachbretter in vielen Fällen symmetrisch.

Eine Position mit dem Turm in "a8", der Dame in "g6" und dem gegnerischen König in "h8" ist genauso verloren wie 
dieselbe Position nur mit dem Turm in "a1", der Dame in "g3" und dem König in "h1".
Dies wäre eine Spiegelung entlang der horizontalen zwischen den Zeilen 4 und 5.
Weiter sind auch Spiegelungen entlang der vertikalen (Zwischen Reihe e und f), den Diagonalen und Rotationen 
(jeweils um 90°, 180° und 270°) möglich.

Durch das simple Spiegeln der Spielsituationen können aus einer validen Spielsituation bis zu sieben weitere ohne
großen Rechenaufwand bestimmt werden. Aus diesem Grund werden in diesem Dokument bei jeder Berechnung neuer Stellungen diese
gespiegelt und die Spiegelungen ebenfalls überprüft und abgespeichert. 

> TODO: Beispielabbildung einfügen

Da Bauern sich nur in eine Richtung bewegen können, gelten für diese andere Regeln beim Spiegeln. 
Insbesondere kann diese Spielsituationen nur eine Spiegelung durchgeführt werden.
Um die Komplexität der Anwendung nicht weiter zu steigern, werden Spielsituationen mit Bauern nicht gespiegelt.

### Funktionen zum Spiegeln

Wie zuvor erklärt, werden in diesem Notebook Stellungen gespiegelt, um die Effizienz der Berechnung zu erhöhen.
Der folgende Abschnitt erklärt die damit verbundende Funktionalität.

Die globale Variable SWAPS enthält für jede Spiegelungsart ein Dictionary. 
In diesem Dictionary liegt für jedes `Square` (Nummeriert von 0-63) eine Zuweisung, welches `Square` der gespiegelten Position entspricht.

So wird beispielsweise bei der vertikalen Spiegelung die Figuren der obersten Reihe mit der untersten Reihe getauscht.
Eine Zuweisung im Dictionary wäre beispielsweise: `0: 56`.   

In [ ]:
SWAPS = {
        "vertical" : {x:x^56 for x in range(64)},
        "horizontal" : {x:x^7 for x in range(64)},
        "rotate_right" : {x:(((x >> 3) | (x << 3)) & 63) ^ 56 for x in range(64)},
        "rotate_180" : {x : x ^ 63 for x in range(64)},
        "rotate_left" : {x : (((x >> 3) | (x << 3)) & 63) ^ 7 for x in range(64)},
        "diagonal" : {x : ((x >> 3) | (x << 3)) & 63 for x in range(64)},
        "anti_diagonal" : {x : (((x >> 3) | (x << 3)) & 63) ^ 63 for x in range(64)}
    }

Die Funktion `get_all_mirror_variations()` berechnet für eine Menge von Stellungen in Integer-Repräsentation (`uniques_int`) die Menge der gespiegelten Variationen inklusive der Original stellungen (`result`).
Hierfür wird die Funktion `mirror_all_directions()` verwendet.

In [ ]:
def get_all_mirror_variations(uniques_int):
    result = uniques_int.copy()
    for unique in uniques_int:
        result |= mirror_all_directions(unique)
    return result

Die Funktion `mirror_all_directions()` erstellt für einen Integer (`board_int`) eine Menge aller Spiegelungen mit den Methoden, die in `SWAPS` aufgeführt sind.
Die Menge `result` enthält den original Integer `board_int` nicht.

In [ ]:
# Implementierung ohne ENUM:
def mirror_all_directions(board_int):
    result = set()
    for name, swap in SWAPS.items():
        result.add(mirror_board(board_int, swap))
    return result

Die tatsächliche Spiegelung wird von der Funktion `mirror_board()` für eine Stellung `board_int` durchgeführt.
Sie speichert das Bit, welches den Spieler am Zug repräsentiert, iteriert über alle Figuren im Integer und führt einen binären Tausch entsprechend der Zuweisungen im `mirror` Dictionary.

> TODO: Methode

In [ ]:
# Mirroring mit 7 Bit Darstellung
def mirror_board(board_int, mirror : dict):
    # Save turn
    result = board_int & 1
    # Get count of pieces saved in int
    n = len(PIECE_LIST)
    # Remove turn
    board_int = board_int >> 1
    
    for i in range(n):
        result |= mirror[board_int & 127] << 7 * i + 1
        board_int = board_int >> 7
    return result

### Hilfsfunktionen für die Ergebnisverwaltung

Die Funktion `add_all_to_used()` fügt die Originale und Spiegelungen aller Stellungen in der Menge `boards_int` zur globalen Variable `USED_BOARDS` hinzu.
Die Funktion überprüft hierbei, ob Spiegelungen durchgeführt werden sollen.
Wenn Spiegelungen durchgeführt werden, wird hierzu die Funktion `add_original_and_mirrors_to_used()` verwendet.

In [ ]:
def add_all_to_used(boards_int):
    global USED_BOARDS
    
    if not USER_WANTS_PAWN:
        for board_int in boards_int:
            add_original_and_mirrors_to_used(board_int)
    else:
        for board_int in boards_int:
            USED_BOARDS.add(board_int)

Die Funktion `add_original_and_mirrors_to_used()` fügt die Integer-Repräsentation einer Stellung, sowie alle Spiegelungen in die globale Variable `USED_BOARDS` ein.
Hierfür werden die zuvor beschrieben Hilfsfunktionen zum Spiegeln verwendet.

In [ ]:
def add_original_and_mirrors_to_used(board_int):
    global USED_BOARDS
    USED_BOARDS.add(board_int)
    if not USER_WANTS_PAWN:
        for reflection in mirror_all_directions(board_int):
            USED_BOARDS.add(reflection)

Da für die Berechnungen temporär Listen von ``Board`` Objekten vorliegen wird die Funktion `multiple_to_integer()` definiert, welche für eine solche Liste `chess_boards` eine Menge der Integer-Repräsentationen (`integers`) erstellt. 

In [ ]:
def multiple_to_integer(chess_boards):
    integers = set()
    for board in chess_boards:
        integers.add(to_integer(board, PIECE_LIST))
    return integers


## Berechnung der $S_n$ Mengen
Die folgende Funktion `calculate()` kann aufgerufen werden, um mit den oben angegebenen Einstellungen eine neue Berechnung durchzuführen. 

Die Funktion berechnet zuerst die Ausgangssituation $S_0$ und speichert sie.
Anschließend werden weitere $n$ in einer Schleife bestimmt.
Nach jeder Berechnung eines $n$, werden die bestimmten Spielsituationen in einer temporären Datei gespeichert. 
Die Funktionen, welche für die Berechnung nötig sind, werden im Verlauf dieses Notebooks aufgeführt und erklärt.

> TODO:
> * Start der Berechnung; start_calculation():
>    * Bekommt Positionen der Figuren übergeben
>    * Berechnung der ersten Menge S_0, damit auch Berechnung der ersten Uniques -> Rückgabe nur Uniques
>        * Hierbei erste wichtige Frage: Werden in used_boards dann ebenfalls nur die Uniques gespeichert oder auch alle?
>        * Schreiben wir die ganze S_0 Menge direkt in die Datei nach der Berechnung?
>    * Schleife iterieren mit altbekannter Bedingung (wenn neue Menge leer oder sich nicht unterscheidet)
>        * 
>        * Berechnung nächster Menge mit Übergabe der Uniques vorheriger Menge

In [ ]:
def calculate():
    count = 0
    sn = calculate_s0()
    print("Done with S" + str(count))
    store_sn(sn)  
    
    while len(sn) != 0:
        sn = calculate_next_sn(sn)
        store_sn(sn)
        count += 1
        print("Done with S" + str(count))
    print("Done")

### Bestimmung von $S_0$

Die Funktion `calculate_s0()` berechnet die Menge aller Stellungen, welche von Weiß gewonnen wurden.
Hierfür wird ein leeres Schachbrett erstellt (`empty_board`) und jede Figur in der `PIECE_LIST` mit der `fill_boards_with_piece` Funktion in allen Konstellationen auf Board-Objekten verteilt.
Die Funktion liefert eine Menge von Integer Repräsentationen der Stellungen in $S_0$. 

In [ ]:
def calculate_s0():
    # TODO: Hier muss noch irgendwo eine Berücksichtigug rein, dass der Bauer durch die Queen beim Erstellen von s0  ersetzt wird, dies muss dazu noch rückgängi gemacht werden, wenn die Queen durch den Pawn ersetzt wird.
    global PIECE_LIST

    s0 = set()
    empty_board = chess.Board(None)
    # Turn has to be black while checkmate
    empty_board.turn = chess.BLACK

    # Create temp_piece_list for conversion between board_object and integer
    tmp_piece_list = []
    s0.add(to_integer(empty_board, tmp_piece_list))
   
    # TODO: Wollen wir das hier haben?
    if USER_WANTS_PAWN: 
        PIECE_LIST[PIECE_LIST.index(chess.Piece.from_symbol('P'))] = chess.Piece.from_symbol('Q')

    for piece in PIECE_LIST:
        s0 = fill_boards_with_piece(s0, piece, tmp_piece_list)
        tmp_piece_list.append(piece)
        
    return s0

Die funktion ``fill_boards_with_piece()`` erstellt aus einer Menge an Stellungen (in Integer-Representation) `int_boards_set` und einer Figur `piece` eine Menge von Stellungen, welche die Positionen in `int_board_sets` an jeder freien Stelle um `piece` ergänzen.
Jedes Mal, wenn eine Figur platziert wird, wird eine Kopie des Board-Objektes erstellt.

Wenn der zweite König platziert wird, wird die Stellung zusätzlich auf Validität überprüft.
Wenn alle Figuren platziert wurden, werden nur Boards, in denen Schwarz matt ist, zurückgegeben. 

Die Funktion benötigt als weiteren parameter die Piece List, welche die Figuren enthält, die in `int_boards_set` verwendet werden.
Diese Liste wird für das Erstellen von Board-Objekten aus Integern und dem Erstellen von Integern verwendet.

In [ ]:
def fill_boards_with_piece(boards_int_set, piece, cur_piece_list):
    finished_boards = set()
    all_squares = set(range(64))
    piece_count = len(PIECE_LIST)
    for int_board in boards_int_set:
        board_o = to_board(int_board, cur_piece_list)
        used_squares = set(board_o.piece_map().keys())
        for square in all_squares:
            if square not in used_squares:
                tmp_piece_list = cur_piece_list.copy()
                board_object = board_o.copy()
                board_object.set_piece_at(square, piece)
                tmp_piece_list.append(piece)
                board_int = to_integer(board_object, tmp_piece_list)
                if len(used_squares) > 1 and not board_object.is_valid():
                    # Don't process invalid boards further than the second king
                    continue

                if board_object.is_checkmate():
                    if board_int not in USED_BOARDS:
                        finished_boards.add(board_int)
                        add_original_and_mirrors_to_used(board_int)
                        continue

                if len(used_squares) + 1 < piece_count: 
                    #Board is valid, but needs more pieces
                    finished_boards.add(board_int)
    return finished_boards

Nachdem die erste Menge $S_0$ berechnet wurde, müssen iterativ alle verbleibenden $n$ bestimmt werden.
Diese Aufgabe übernimmt die Funktion `calculate_next_sn()`. Ausgehend von einer Menge an Integern `sn` wird die Menge $S_{n+1}$ (`sn_p_1`) bestimmt. 

Hierfür wird über jede Stellung in der Menge `sn` iteriert, relevante Spielzüge ausgeführt (unterschieden zwischen Situationen mit Bauern und ohne), duplikate entfernt und die erreichten Situationen sowohl der Menge $S_{n+1}$ als auch der globalen Variable `USED_BOARDS` hinzugefügt.
Ist $n$ gerade ($n = 2i$), wird mit der Funktion `check_black_determinism` weiter überprüft, ob alle Spielzüge in einer Menge  $S_m$ mit $m < n$ enden.

> TODO: Keep if valid vielleicht hier?

In [ ]:
def calculate_next_sn(sn):
    sn_p_1 = set()
    for board_int in sn:
        # Nicht sicher, ob hier schon die Bretterstellung notwendig ist (glaube nicht)
        chess_board = to_board(board_int, PIECE_LIST)
        chess_board.turn = chess_board.turn ^ True

        # Berücksichtigung bei dieser Trennung, dass irgendwann auch der Pawn in eine Queen umgewandelt wird und dann muss USER_WANTS_PAWN = False sein -> Danach aber vorsichtig bei Spiegelungen sein, weil diese auch nicht instant verwendet werden können -> Möglicherweise doch so umsetzbar
        
        if need_pawn_moves(chess_board):
            reached_boards = moves_with_pawns(chess_board)
        else:
            reached_boards = moves_without_pawns(chess_board)
            
        if not chess_board.turn:
            reached_boards = check_black_determinism(reached_boards)
        
        reached_integers = multiple_to_integer(reached_boards)
        
        # Remove unwanted mirrors
        reached_integers = reached_integers - USED_BOARDS
        
        sn_p_1 |= reached_integers
        add_all_to_used(reached_integers)

    return sn_p_1

### Berechnung der Spielzüge für Stellungen ohne Bauern

Die Funktion `moves_without_pawns()` führt für ein Objekt `chess_board` alle möglichen Züge der Figuren aus.

Da die Rechnungen rückwärts (von $S_0$ zu $S_n$ mit $n > 0$) durchgeführt werden, muss der Spieler am Zug vor dem Hinzufügen zur temporären Ergebnisliste `tmp` gespiegelt werden.

Die Funktion überprüft alle entstehenden Situationen auf Validität und gibt die validen Stellungen `reached_boards` zurpck.

In [ ]:
def moves_without_pawns(chess_board):
    tmp = []
    for pLMove in chess_board.pseudo_legal_moves:
        # Backwards-execuion of Move
        chess_board.push(pLMove)
        chess_board.turn = chess_board.turn ^ True
        tmp.append(chess_board.copy())
        chess_board.turn = chess_board.turn ^ True
        chess_board.pop()
        
    reached_boards = keep_if_valid(tmp)  

    return reached_boards

### Berechnungen der Spielzüge für Stellungen mit Bauern

Die Funktion `moves_with_pawns()` überprüft, ob sich ein Bauer auf dem Spielfeld (im Objekt `chess_board`) befindet.
Befindet dieser sich in der Stellung, wird er ein (oder zwei, wenn in Reihe 4) Feld(er) nach hinten bewegt.
Befindet sich kein Bauer in der Stellung befindet sich eine Dame in der obersten Reihe (zuvor durch ``need_pawn_moves`` überprüft). In diesem Fall wird die Dame durch einen Bauern in der 7. Reihe ersetzt.

Valide Spielsituationen werden als Liste von Objekten `reached_boards` zurückgegeben.

In [ ]:
def moves_with_pawns(chess_board):    
    if check_for_pawn(chess_board):
        tmp = move_pawn(chess_board)
    else:
        tmp = [replace_queen(chess_board)]
    
    reached_boards = keep_if_valid(tmp)

    return reached_boards

Die Funktion `check_for_pawn()` überprüft, ob sich Bauern in einer Stellung `chess_board` befinden.

In [ ]:
def check_for_pawn(chess_board):
    return len(set(chess_board.pieces(chess.PAWN, chess.WHITE))) > 0

Die Funktion ``need_pawn_moves()`` überprüft, ob Bauern-Züge für eine Situation `chess_board` beachtet werden müssen.
Hierfür verwendet sie das `USER_WANTS_PAWN` Flag, die Farbe am Zug, und ob sich Bauern in der Stellung oder eine Dame in der obersten Reihe befinden.

In [ ]:
def need_pawn_moves(chess_board):
    return USER_WANTS_PAWN \
           and chess_board.turn \
           and (check_for_pawn(chess_board) or check_top_row_for_queen(chess_board))

Die Funktion `find_pawn()` gibt die Nummer des Feldes zurück, in welchem sich ein Bauer auf der Stellung in `chess_board` befindet.

In [ ]:
def find_pawn(chess_board):
    return chess_board.pieces(chess.PAWN, chess.WHITE).pop()

Die Funktion `move_pawn()` setzt einen Bauern in der Situation `chess_board` auf ein Feld in einer niedrigeren Reihe.

Befindet sich der Bauer in der vierten Reihe, wird er sowohl in die dritte als auch zweite Reihe gesetzt.
In den Reihen 3,5,6 und 7 wird er jeweils eine Reihe zurück gesetzt.

Die Rückgabe `res` enthält demnach 0 - 2 Board Objekte.

In [ ]:
def move_pawn(chess_board):
    res = []
    position = find_pawn(chess_board)
    if position in range(24-31):
        tmp_board = chess_board.copy()
        tmp_board.remove_piece_at(position)
        tmp_board.set_piece_at(position - 16, chess.Piece.from_symbol('P'))
        res.append(tmp_board)
    
    if position in range(16-63): 
        chess_board.remove_piece_at(position)
        chess_board.set_piece_at(position - 8, chess.Piece.from_symbol('P'))
        res.append(chess_board)
    
    return res

Die Funktion `check_top_row_for_queen()` überprüft für eine Situation `chess_board`, ob sich in der 8. Reihe eine Dame befindet.
Die Rückgabe erfolgt als boolscher Wert.

In [ ]:
def check_top_row_for_queen(chess_board):
    square_with_queen = chess_board.pieces(chess.QUEEN, chess.WHITE).pop()
    return square_with_queen in range(56,64)

Die Funktion `replace_queen()` ersetzt in einer Situation `chess_board` eine Dame in der obersten Reihe durch einen Bauern in Reihe 7.

In [ ]:
def replace_queen(chess_board):
    queen_square = chess_board.pieces(chess.QUEEN, chess.WHITE).pop()
    chess_board.remove_piece_at(queen_square)
    chess_board.set_piece_at(queen_square - 8, chess.Piece.from_symbol('P'))
    
    return chess_board

## Hilfsfunktionen für die Rechnung

Die folgenden Funktionen werden verwendet, um die vorher definierten Funktionen zu unterstützen.

Die Funktion `keep_if_valid()` sortiert aus einer Liste `chess_boards` die nicht validen Stellungen aus und gibt die validen als `res` zurück.

In [ ]:
def keep_if_valid(chess_boards):
    res = []
    for board in chess_boards:
        if board.is_valid():
            res.append(board)    
    return res

Wenn mittels der KI eine Spielsituation ausgewertet wird, kann für jeden Zug des weißen Spielers ein Zug ausgewählt werden.
Für die Stellungen, bei denen Schwarz am Zug ist, muss die KI alle möglichen Züge auswerten können.
Da jedoch für einen spezifischen Zug, welcher eine Stellung von $S_n$ in $S_{n-1}$ führt, dasselbe nicht für alle Züge gilt, welche in der Stellung möglich sind, müssen die Stellungen, bei welchen Schwarz am Zug ist, besonders gefiltert werden.
Für jede Stellung $b$ aus einem $S_n$ mit $n \, \% \, 2 = 0$ muss folglich gelten:  
$$
b \in S_n \implies \forall m \in valid\_moves(b): b.push(m) \in S_{m} \land m < n
$$
Wobei `valid_moves` die Liste der legalen Züge für eine Stellung ist und `b.push(m)` die Stellung beschreibt, welche durch Ausführen des Zuges $m$ entsteht. 

Die Funktion `check_black_determinism` stellt dies sicher.
Für jede Stellung in `chess_boards` wird jeder mögliche legale Zug ausgeführt und überprüft, ob die entstehende Stellung in einer Menge
$S_m$ mit $m <= n$ auffindbar ist. Nur wenn alle Züge diese Bedingung erfüllen, wird das Objekt in die Liste `deterministic` aufgenommen und zurückgegeben.

In [ ]:
def check_black_determinism(chess_boards):
    deterministic = []
    for chess_board in chess_boards:
        include = True
        for move in chess_board.legal_moves:
            chess_board.push(move)
            rep = to_integer(chess_board,PIECE_LIST)
            chess_board.pop()
            if rep not in USED_BOARDS:
                include = False
                break
        
        if include:
            deterministic.append(chess_board)
            
    return deterministic

### Funktionen für die Ergebnispersistierung

> TODO: Optimieren / Erklären von Speicherkonzept

Um nach jeder abgeschlossenen Berechnung eines $S_n$ dieses Ergebnis an eine Datei anfügen zu können wird zunächst eine temporäre leere Datei erstellt.
Die Funktion `create_empty_file()` erfüllt diese Aufgabe. 

In [ ]:
def create_empty_file():
    f = open("S_n_Results/" + FILENAME + ".preConvert", "w")
    f.write("")
    f.close()

Die Funktion `store_sn` fügt die Inhalte der Menge `sn` an das Ende der temporären Datei an.

In [ ]:
def store_sn(sn):
    f = open("S_n_Results/" + FILENAME + ".preConvert", "a")
    f.write("\n")
    f.write(json.dumps(list(sn)))
    f.close()

Nach erfolgreicher Berechnung aller $S_n$ wird die Funktion `convert_temp_file()` verwendet, um eine finale `.chessAI` Datei zu erstellen.

Hierfür wird die temporäre `.preConvert` Datei wieder eingelesen, und eine Liste mit allen errechneten Mengen sowie der ``PIECE_LIST`` erstellt.
Diese Liste wird mittels Pickle in eine `.pickle` Datei gespeichert, welche anschließend gezippt in einer `.chess.AI` Datei gespeichert werden.
Letztlich werden temporäre Daten gelöscht.

In [ ]:
def convert_temp_file():
    f = open("S_n_Results/" + FILENAME + ".preConvert", "r")
    lines = f.readlines()
    
    data = [PIECE_LIST]
    
    for line in lines[1:]:
        list = json.loads(line)
        data.append(set(list))
        
    f = open("S_n_Results/" + FILENAME + ".pickle", "wb")
    f.write(pickle.dumps(data))
    f.close()
    
    with ZipFile("S_n_Results/" + FILENAME + '.chessAI', 'w', compression=ZIP_DEFLATED) as zipped:
        zipped.write("S_n_Results/" + FILENAME + ".pickle", FILENAME + ".pickle")
    if os.path.exists("S_n_Results/" + FILENAME + ".chessAI") and os.path.exists("S_n_Results/" + FILENAME + ".pickle"):
        os.remove("S_n_Results/" + FILENAME + ".pickle") 

    if os.path.exists("S_n_Results/" + FILENAME + ".preConvert"):
        os.remove("S_n_Results/" + FILENAME + ".preConvert")

Ausführen der Berechnung

In [ ]:
%%time
create_empty_file()
calculate()
convert_temp_file()
